<a href="https://colab.research.google.com/github/branndonm1/branndonm1/blob/main/RandomWalkProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stating the problem in words


Supposed that we live on a plane where we can move in four possible directions -- north, south, east, and west. If on any particular walk we end up a distance of more than 4 units away from where we started, we must pay a fine. What is the longest walk we could take so that on average we do not have to pay the fee? 


# Writing the problem in math

Lets use some math to solve this problem. We of course have to make a model and then apply the concepts we know to analyze the model. 

Model assumptions: 
- Starting point is the origin. 
- We can only take whole number steps in any of the four possible directions.
- The distance between the starting point and end point $(x,y)$ is given by $d = \sqrt{x^2+y^2}$.

Definitions:
- We will say that a walk is a path of length $t$ steps.  
- A path of length $t$ steps is a particular arrangement of $t$ sequential choices from the set $\{N,S,E,W\}$
- The end point of a path is the coordinates in the plane we end up at after taking the $t$ choices. 
- Note: several distinct walks could have the same end point. For example the walk $W_1 = NE$ and the walk $W_2=EN$ are distinct, but they have the same end point in the plane $(1,1)$






# Mathematical Analysis


There are several approaches one can take to try and solve this problem given our setup. I outline two ways in the next sections. 

# 1.) Combinatorial Approach


Once can try and take a simple counting approach (which is what I initially did): 

We can simply count the total number of paths. Then count the number of paths who's end points are at most a distance of 4 from origin. We can write the probability that you don't pay the toll at time $t$ is:

$$P(t) = \frac{|\{\text{paths with end point distance less than or equal to 4 at time t}\}|}{|\{\text{paths at time t}\}|} \hspace{.3cm}:= \frac{A(t)}{B(t)}.$$

So our goal is to find the largest $t$ such that $P(t) > .5$. Of course part of the problem is finding a formula for $P(t)$ for us to work with but lets try make our lives easier by finding $A(t)$ and $B(t)$ individually. We can count the total number of unique paths after taking $t$ step, and get $B(t) = 4^t$. This is because after each step, we have four new directions we can move in. Now for a given time we need to now all the paths than end up with and endpoint at most a distance 4 from the origin. With a little bit of counting (see way below) we can deduce the total number of end points that are at most distance 4 from the origin. The problem is that there is no clear way to find the number of paths after taking $t$ steps that end at a specific end point. Things get complicated because paths can double back. For example, for a path of length 2, there are four paths that end up at the origin by taking one step in any direction then stepping back. Now for paths of length 100, its not so clear how to count the number of total paths that end at the origin -- we might take 50 steps east then some back, or 1 step east and back a total of 50 times. There are many other possible paths that will end at the origin. There is also no symmetry to exploit since other end points might only have 1 single path that reaches them. For example, for a walk of length $t$, the the end point (0,100) will only have one path that reaches it. All this is to say there is no clear way to count the paths that will result in an end point whos distance is at most 4 for a general path of length $t$.

# 2.) Random Variable Approach

Lets try and take a random variable approach (which ends up being more fruitful): 

Notice that all that determine your final end point after $t$ steps is how many forward and backward steps we took in each dimension. Lets define the random variables 

$$F_i(t)=\text{number of forward steps in dimension d out of the total of $t$ steps}.$$ 
$$B_i(t)=\text{number of backward steps in dimension d out of the total of $t$ steps}.$$ 

We learned from the combinatorial approach that we need to find some relationship between the possible end points $(x,y)$ and the number of paths of length $t$ that end at $(x,y)$. Notice that the final point we end up in is just the difference of the forward and backward steps. For example, 

$$F_1 = 5, B_1 = 1, F_2=2, B_2=4$$

corresponds to the end point $x$-coordinate

$$(x,y) = (F_1-B_1, F_2-B_2)=(5-1,2-4)=(4,-2)$$

So the random variable for the $i$th coordinate of final end point is $$Z_i(t) := F_i(t)-B_i(t).$$
What we want to do is find the largest $t$ so that 

$$\mathbb{E}[\sqrt{Z_1(t)^2+Z_2(t)^2}] \leq 4$$

It is clear that for $t$ steps, the variables $\{F_1,B_1,F_2,B_2\}$ follow a multinomial distribution with parameters $n=t$ and $p_i=t/
4$. For large enough $t$, the multinomial distribution approaches a distribution of joint i.i.d poisson random variables each with paramater $\lambda_i = \frac{t}{4}$. But recall that for large $t$ we can approximate the poisson distribution with the normal distribution $N(\lambda, \lambda)$. This means that we can approximae $Z$ as the difference of normal variables to get

$$Z_i = F_i-B_i \sim N(\lambda_i, \lambda_i)-N(\lambda_i, \lambda_i) = N(0,2\lambda_i)=N(0,\frac{t}{2}).$$

Now the square root of a sum of $k$ normal $N(0,\sigma^2)$ variables is $\chi_k$ distributed. So given our past approximation get 

$$\sqrt{Z_1^2+Z_2^2} \sim \chi_2.$$

We know the expected value of this $\chi_2$ explicitly so we can apprximate

$$\mathbb{E}[\sqrt{Z_1^2+Z_2^2}] \approx \mathbb{E}[\chi_2] = \sqrt{2\sigma^2}\frac{\Gamma(\frac{3}{2})}{\Gamma(\frac{2}{2})} = \sqrt{2\lambda}\frac{\sqrt{\pi}}{2}= \sqrt{\frac{2\frac{t}{2}\pi}{4}}=\frac{\sqrt{t\pi}}{2}.$$ 

Now we can use the explicit formula to solve for the largest $t$ that will yield expected path less than or equal to 4. We solve the inequality $\frac{\sqrt{t\pi}}{2} \leq 4$ to get that we must have $t \leq 20$. This means we can take path of maximum 20 steps so that on average we will not have to pay the fine. 

# Simultation

Lets code up an experiment that will (hopefully) confirm our pen and paper work. 


In [1]:
import random

In [4]:
NUM_OF_EXP = 20000
MAX_WALK_LENGTH = 55

def random_walk(n):
  "return coodinates after taking n random steps"
  x, y = 0, 0
  for i in range(n):
    dx, dy = random.choice([(0,1), (0,-1), (1,0), (-1,0)]) 
    x+=dx
    y+=dy
  return (x,y)

for walk_length in range(1,MAX_WALK_LENGTH+1):
  no_fee_counter = 0
  for i in range(1,NUM_OF_EXP+1):
    (x,y) = random_walk(walk_length)
    d = (x**2+y**2)**.5
    if d <= 4:
      no_fee_counter+=1
    
  print("The percentage of walks of length ", walk_length, "that will be free is:", 100*no_fee_counter/NUM_OF_EXP, "%")


The percentage of walks of length  1 that will be free is: 100.0 %
The percentage of walks of length  2 that will be free is: 100.0 %
The percentage of walks of length  3 that will be free is: 100.0 %
The percentage of walks of length  4 that will be free is: 100.0 %
The percentage of walks of length  5 that will be free is: 95.7 %
The percentage of walks of length  6 that will be free is: 93.82 %
The percentage of walks of length  7 that will be free is: 88.55 %
The percentage of walks of length  8 that will be free is: 86.595 %
The percentage of walks of length  9 that will be free is: 81.065 %
The percentage of walks of length  10 that will be free is: 79.185 %
The percentage of walks of length  11 that will be free is: 74.43 %
The percentage of walks of length  12 that will be free is: 73.05 %
The percentage of walks of length  13 that will be free is: 67.845 %
The percentage of walks of length  14 that will be free is: 67.495 %
The percentage of walks of length  15 that will be fr

Notice that the largest length $t$ where at least 50% of the walks are at most a distance of 4 away from the origin is $t \approx 22$. This means that experimentally, we can take a walk of at most 22 steps so that on average we do not have to pay the fine. This result is consistent with our pen and paper work.  